# 1. Carregamento das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as scs

# 2. Exploração dos dados

Este capítulo do Notebook apresenta como foram tratados os dados para posterior análise.

## 2.1. Conhecendo o DF, prévia e tipo dos dados

In [ ]:
#Conhecendo o DF
df= pd.read_csv('dados_finais.csv')

print('shape:', df.shape)


In [ ]:
#Conhecendo uma prévia dos dados
df.head(10)

In [ ]:
#Setando o ID como index do DF
df.set_index('ID', inplace=True, drop=True)

In [ ]:
#checando os nomes das colunas
df.columns 

In [ ]:
#checando os tipos dos dados
df.dtypes

In [ ]:
df.describe()

## 2.2. Verificação de dados duplicados, checagens e tratamentos

In [ ]:
# Verificando se há dados de index duplicados
df[df.index.duplicated()==True]

In [ ]:
#validando se o número total de homens+mulheres bate com o total da população
df[(df['HOMEMTOT']+df['MULHERTOT']) !=df['TPM']]

In [ ]:
#verificando se as colunas da raspagem de dados há dados duplicados:
colunas=['TPM', 'DDM', 'SMM',
       'POM', 'TPO', 'RSA', 'TXE', 'IDEBI', 'IDEBF', 'PIB', 'RFE', 'RER',
       'DEM', 'OPNV', 'IMH', 'AUT', 'ESA', 'AVP', 'UVP']
df[df.duplicated(colunas)==True]

Não foram verificados dados duplicados e/ou discrepantes entre as bases pesquisadas (Atlas Brasil e IBGE).

## 2.3. Verificação de dados faltantes

In [ ]:
#verificando dados faltantes (por atributo)
nulos=df.isna().sum()

In [ ]:
plt.barh(nulos[nulos>0].index,nulos[nulos>0])
plt.show()

In [ ]:
print(nulos[nulos>0])

In [ ]:
#Total de registros faltantes no dataset
df.isna().sum().sum()

Como alguns atributos apresentaram altas taxas de dados faltantes, optou-se por exclui-los do dataset ao invés de preenchê-los com a média ou moda para não criar vieses. Sendo assim, os atributos "OPNV", "RFE" e "IMH" foram retirados do dataset:

In [ ]:
df_original=df.copy()

df.drop(columns=['OPNV','RFE','IMH'], inplace=True)

Para os demais dados faltantes, o procedimento realizado foi o preenchimento de acordo com a média dos valores de cada atributos.

In [ ]:
#preenchendo os dados faltantes com a média dos valores:
df.fillna(df.mean(),inplace=True)

In [ ]:
print("Total de dados faltantes após o tratamento:", df.isna().sum().sum())
print(df.isna().sum())

In [ ]:
#Descrição da coluna de Score (GGI):
df['SCO'].describe()

## 2.5. Padronização dos dados

Com um dataset composto por diversos atributos de diferentes grandezas, outro tratamento necessário é a padronização dos dados. A padronização foi realizada em uma parte do dataset, que desconsidera os atributos categóricos.

In [ ]:
from sklearn.preprocessing import StandardScaler


# Separando os valores no Dataset
p = df.iloc[:,4:]
colunas=p.columns


# padronizando os atributos
p = StandardScaler().fit_transform(p)

print(p)


In [ ]:
df_p=pd.DataFrame(p, columns=colunas, index=df.index)

# 3. Aplicação dos modelos

Neste capítulo são apresentadas as aplicações dos algoritmos, métricas e otimizações utilizadas.

## 3.1. Otimização de hiperparâmetros

In [ ]:
from sklearn import cluster, metrics

In [ ]:
def best_cluster(algorithm_name, X, upK,downK=1):
    list_n_k=[]
    list_silh=[]
    
    silh_max=-1e12
    k_silh_max=0
    
    if algorithm_name=='KMeans':
        algorithm=cluster.KMeans(n_clusters=1)
        
    elif algorithm_name=='Ward':
        algorithm=cluster.AgglomerativeClustering(n_clusters=1, linkage='ward')
        
    else: 
        print('Algoritmo inválido')
        return 0
    
    for iterator in range(downK,upK+1):
        
        if iterator==1:
            list_n_k.append(1)
            list_silh.append(0)
            continue
        
        algorithm.n_clusters=iterator 
        
        cluster_labels=algorithm.fit_predict(X)
        
        list_n_k.append(iterator)
        silh=metrics.silhouette_score(X,cluster_labels)
        list_silh.append(silh)
        
        if silh>silh_max:
            silh_max= silh
            k_silh_max=iterator
    
    
    plt.figure()
        
    plt.title("Busca do silhouette ótimo - "+algorithm_name)
    plt.grid()
        
    plt.plot(list_n_k, list_silh) 
        
    return print(algorithm_name," - Número de clusters ideais considerando a métrica silhouette: ", k_silh_max)

### 3.1.1 - Número de Clusters: K-Means 

Utilizando a métrica Silhouette para avaliação do número ideal de clusters no algoritmo KMeans, temos que o melhor valor encontrado sugere um k=3

In [ ]:
best_cluster("KMeans",p, 15)

Outra forma de avaliação para o número de clusters é a avaliação através do "Método Elbow", que avalia o somatório dos erros quadráticos das instâncias de cada cluster. Neste método, podemos observar que o número de clusters ideais também sugere k=3.

In [ ]:
from sklearn.cluster import KMeans
wcss = []
 
for i in range(1, 15):
    kmeans = KMeans(n_clusters = i, random_state=10)
    kmeans.fit(p)
    print (i,kmeans.inertia_)
    wcss.append(kmeans.inertia_)  
plt.plot(range(1, 15), wcss)
plt.title('O Metodo Elbow')
plt.grid()
plt.xlabel('Numero de Clusters')
plt.ylabel('WCSS')
plt.show()

### 3.1.2 - Número de Clusters: Clustering Hierárquico

Outra abordagem é a utilização de algoritmos de clustering hierárquico. Neste trabalho utilizou-se a abordagem aglomerativa com o critério "Ward", que minimiza a variância dos clusters mesclados. 
Ao aplicarmos a otimização de hiperparâmetros de acordo com a métrica silhouette, verificou-se que o número de clusters ótimo é 3.

In [ ]:
best_cluster("Ward",p, 15)

## 3.2. Aplicação do K-Means para dados padronizados

Com as avaliações no item 3.1.1, foi identificado que o número de clusters ideais para aplicação do KMeans neste dataset é 3.

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state=10)

kmeans.fit(p)

In [ ]:
distance = kmeans.fit_transform(p)

In [ ]:
labels = kmeans.labels_
pl=labels
pl

Observando o resultado do algoritmo, temos que somente duas instâncias foram agrupadas no cluster 2:

In [ ]:
df_kmeans=df.copy()
df_kmeans['Cluster']=pd.Series(pl).values
df_kmeans['Cluster'].value_counts()

As duas instâncias que foram agrupadas no cluster 2 foram Rio de Janeiro e São Paulo:

In [ ]:
df_kmeans[df_kmeans['Cluster']==2][['UF','NOMEMUN','Cluster']]

Podemos observar tambem que regiões Norte e Nordeste possuem a maioria dos seus municípios classificados como Cluster 0, enquanto que nas demais regiões, a maioria dos municipios foi classificada como Cluster 1

In [ ]:
df_kmeans[['REGIAO','Cluster','NOMEMUN']].groupby(['REGIAO','Cluster']).count()

In [ ]:
df_kmeans.groupby(['REGIAO','UF','Cluster']).size().unstack().plot(kind='bar',stacked=True)
plt.title('Quantidade de cidades por cluster em cada estado')
plt.ylabel('Quantidade de cidades')
plt.xlabel('Estados brasileiros')
plt.tight_layout()
plt.show()

Aplicação do PCA para redução de dimensionalidade e visualização dos dados:

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
P=pca.fit_transform(p)

print('Soma da variância acumulada:{}'.format(pca.explained_variance_ratio_ .sum()))
P

Com a redução da dimensionalidade para n=2, temos uma variância acumulada de aproximadamente 60% dos dados. Para uma representação melhor, precisariamos de um n maior mas não seria interessante para observação gráfica.

In [ ]:
df_k_pca=pd.DataFrame(P, columns=['PC1', 'PC2'])
df_k_pca['Cluster']=pl
df_k_pca['Cidade']=df_kmeans['NOMEMUN'].values
df_k_pca

In [ ]:
from sklearn import cluster, metrics

kmeans.fit(P)

kmeans.cluster_medoids_=[]

medians_index, _ = metrics.pairwise_distances_argmin_min(kmeans.cluster_centers_ , P[:,:2])

medians_index #array com os indices das instancias dos clusters

for m in medians_index:
    kmeans.cluster_medoids_.append(np.array( [P[m,0],P[m,1]] ))

kmeans.cluster_medoids_

labels=kmeans.labels_
centers=kmeans.cluster_centers_
medoids=np.array(kmeans.cluster_medoids_)


plt.figure(figsize=(10,8))


plt.scatter(P[:,0],P[:,1], c=labels, label="Instâncias")


plt.scatter(centers[:,0],centers[:,1], marker='X', c=np.unique(labels),s=200, ec='k', alpha=0.6, label='Centróide')
plt.scatter(medoids[:,0],medoids[:,1], marker='o', c="None",s=100, ec='k', alpha=0.6, label="Medóide")

plt.legend()

Identificando os medóides de cada cluster, temos as cidades de AQUIDABÃ (Sergipe), Conceição das Alagoas (MG) e São Paulo (SP), como instâncias mais próximas dos clusters 0, 1, 2 respectivamente.

In [ ]:
df_k_pca[df_k_pca['PC1'].isin(medoids[:,0])]

Além disso, podemos verificar os principais atributos em cada Componente:

In [ ]:
a={key:[x,y] for key,x,y in zip(colunas,pca.components_[0],pca.components_[1])}

rank_pc1 = [(colunas, pca_components_[0]) for colunas, pca_components_ in sorted(a.items(), 
                                                                                 key=lambda x: abs(x[1][0]), reverse=True)]

rank_pc2 = [(colunas, pca_components_[1]) for colunas, pca_components_ in sorted(a.items(), 
                                                                                 key=lambda x: abs(x[1][1]), reverse=True)]

In [ ]:
for i in range(4):
    print("PC1 - Atributo",i+1,":",rank_pc1 [:4][i])

print('-'*60)

for i in range(4):
    print("PC2 - Atributo",i+1,":",rank_pc2 [:4][i])

## 3.3. - Aplicação de Clustering Hierárquico em dados padronizados: Ward

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
w=AgglomerativeClustering(n_clusters=3, linkage='ward')

w.fit_predict(p)

labels_ward=w.labels_

In [ ]:
df_ward=df_p.copy()
df_ward['Cluster']=pd.Series(labels_ward).values
df_ward['Cluster'].value_counts()

In [ ]:
df_ward['Cidade']=df['NOMEMUN']
df_ward['Cidade'][df_ward['Cluster']==2]

In [ ]:
df_ward_pca=pd.DataFrame(P, columns=['PC1', 'PC2'])
df_ward_pca['Cluster']=df_ward['Cluster'].values
df_ward_pca['Cidade']=df_ward['Cidade'].values
df_ward_pca

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x=df_ward_pca['PC1'], y=df_ward_pca['PC2'], hue=df_ward_pca['Cluster'])

In [ ]:
df_ward['REGIAO']=df['REGIAO']
df_ward['UF']=df['UF']
df_ward.groupby(['REGIAO','UF','Cluster']).size().unstack().plot(kind='bar',stacked=True)
plt.title('Quantidade de cidades por cluster em cada estado - WARD')
plt.ylabel('Quantidade de cidades')
plt.xlabel('Estados brasileiros')
plt.tight_layout()
plt.show()